# Benchmarking with sktime

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sktime.benchmarking import Orchestrator
from sktime.benchmarking import Data
from sktime.benchmarking import Results
from sktime.benchmarking import Evaluator
from sktime.benchmarking.metrics import ScoreAccuracy

from sktime.highlevel import TSCTask 
from sktime.highlevel import TSCStrategy

from sktime.datasets import load_gunpoint
from sktime.datasets import load_arrow_head

from sktime.classifiers.ensemble import TimeSeriesForestClassifier
from sktime.classifiers.time_series_neighbors import KNeighborsTimeSeriesClassifier

import os
import pandas as pd
import numpy as np

### Set up orchestration
1. Initialise `Data` object which encapsulates all the necessary information about the data, including whether there exist predefined train and test folds, and iteratively loads the data during orchestration
2. Specify the tasks of interest, one for each dataset with the corresponding name of the target variable to be predicted
3. Specify the prediction strategies to evaluate 
4. Specify where to save results

In [3]:
datadir = 'sktime/datasets/data/'
datasets = os.listdir(datadir)
print('Datasets: ', datasets)
n_datasets = len(datasets)
data = Data(datadir=datadir, 
            names=datasets, 
            train_test_exists=True)

Datasets:  ['ItalyPowerDemand', 'ArrowHead', 'GunPoint']


In [4]:
# in this example, the target name is the same for all datasets
tasks = [TSCTask(target='class_val') for _ in range(n_datasets)]

In [5]:
strategies = [
    ('tsf10', TSCStrategy(TimeSeriesForestClassifier(n_estimators=10))),
    ('tsf20', TSCStrategy(TimeSeriesForestClassifier(n_estimators=20)))
]

In [6]:
results_dir = '../sktime-benchmarking-results/'

### Run benchmarking

In [11]:
orchestrator = Orchestrator(data=data,
                            tasks=tasks, 
                            strategies=strategies,
                            cv='presplit', 
                            results_dir=results_dir)
results = orchestrator.run()

Running strategies on ItalyPowerDemand
Running strategies on ArrowHead
Running strategies on GunPoint


In [8]:
results.__dict__

{'results_dir': '../sktime-benchmarking-results/',
 'dataset_names': ['ItalyPowerDemand', 'ArrowHead', 'GunPoint'],
 'strategy_names': ('tsf10', 'tsf20')}

In [81]:
os.listdir(os.path.join(results_dir, 'tsf10', 'ItalyPowerDemand'))

['test0.csv']